## Examples for simple Quantum Circuits

We need the QCSim backend.

In [1]:
%run QCSim_Backend.ipynb

As a first step, we take a close look at how the ground state $\left\lvert 0 \right \rangle$ of a single qubit is encoded. Since QCSim uses density operators, this is a 2x2-matrix.

In [2]:
DOp.GroundState(1)

array([[1., 0.],
       [0., 0.]])

As expected, this matrix is the orthogonal projection onto the subspace $\mathbb{C} \left\lvert 0 \right \rangle$ of the 2-dimensional 1-qubit Hilbert space $\mathbb{H} = \mathrm{span}\left\{ \left\lvert 0 \right \rangle, \left\lvert 1 \right \rangle \right\}$.

We can always calculate the probability to measure a given qubit in a multi-qubit system in a given state. Of course, the probability to measure the state $\left\lvert 0 \right \rangle$ in the state $\left\lvert 1 \right \rangle$ is zero:

In [3]:
DOp.MeasureProb(DOp.GroundState(1), 0, 1)

0.0

#### Circuits
Quantum circuits are commonly represented by diagrams, which are read from left to right. Here, the reading direction is from top to bottom.
The diagrams are drawn as ASCII-pictures. For example, a single vertical line ("|") stands for the identity gate on a single qubit system.

In [4]:
circuit = QCirc.FromString("""
|
""")

Circuits, once defined in terms of a diagram, can be simulated. For this, an initial state has to be specified: In most cases, this will be the ground state.

In [5]:
resultStack = circuit.simulate(DOp.GroundState(1))
resultStack

[(1.0,
  array([[1.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]]))]

The output of the simulation is a so-called <i>stack</i> of density operators. This is a list of pairs $(p,\Phi)$, where $p$ is the probability that the system after the simulation is in the state defined by the density operator $\Phi$.
In our case, the stack has only one entry, which means that after the simulation, the probability to still find the system in the ground state is 1.

To obtain a single density operator, we can always <i>meld</i> a stack. The result is a weighted sum of the members of the stack.
In our case, the function DOp.MeldStack returns the density operator $$\mathrm{meldOp} = \left(\begin{array} 11 & 0 \\ 0 & 0 \end{array}\right),$$ which we know corresponds to the state $\left\lvert 0 \right\rangle$. Accordingly, the probability to measure this system in the state $\left\lvert 1 \right\rangle$ is zero:

In [6]:
meldOp = DOp.MeldStack(resultStack)
DOp.MeasureProb(meldOp, 0, 1)

0.0

The next simplest circuit consists of a single $X$-gate (or "NOT-gate"), which inverts a qubit. 
In this circuit, we start from the ground state $\left\lvert 0 \right\rangle$, and find the system after the simulation in the state $\left\lvert 1 \right\rangle$ with probability 1.

In [7]:
circ_invert = QCirc.FromString("""
|
X
|
""")

result_inv = DOp.MeldStack(circ_invert.simulate(DOp.GroundState(1)))
DOp.MeasureProb(result_inv, 0, 1)

1.0

Yet another example of a single-gate one-qubit circuit is given by the Hadamard gate. 
After running the simulation, we may measure the system in either states $\left\lvert 0 \right\rangle$ or $\left\lvert 1 \right\rangle$ with equal probabilities:

In [8]:
circ_had = QCirc.FromString("""
|
H
|
""")

result_had = DOp.MeldStack(circ_had.simulate(DOp.GroundState(1)))
DOp.MeasureProb(result_had, 0, 1)

0.5000000000000001

#### 2-Qubit circuits
The $CX$-gate (controlled $X$, or controlled NOT) is a 2-qubit gate which inverts the qubit marked with $x$ iff the control qubit marked with $o$ is in the state $\left\lvert 1 \right\rangle$.
Hence, in the following example, the gate acts as the identity, since we start with the ground state. 

In [9]:
circ_cx = QCirc.FromString("""
| |
x-o
| |
""")

result_cx = DOp.MeldStack(circ_cx.simulate(DOp.GroundState(2)))
DOp.MeasureProb(result_cx, 0, 1)

0.0

If, however, we first invert the control qubit (qubit number 1) then we find that after the simulation, qubit number 0 is in the state $\left\lvert 1 \right\rangle$:

In [10]:
circ_Xcx = QCirc.FromString("""
| X
x-o
| |
""")

result_Xcx = DOp.MeldStack(circ_Xcx.simulate(DOp.GroundState(2)))
DOp.MeasureProb(result_Xcx, 0, 1)

1.0

It is also possible to combine the Hadamard and $CX$-gates to prepare a pair of qubits in a Bell state. 

In [11]:
circ_bell = QCirc.FromString("""
| H
x-o
| |
""")

result_bell = DOp.MeldStack(circ_bell.simulate(DOp.GroundState(2)))
DOp.StateForm(result_bell)

array([0.70710678, 0.        , 0.        , 0.70710678])

In the example above, the function DOp.StateForm has been used, which attempts to reconstruct a state vector from a density operator. The resulting array is to be interpreted as the list of coefficients for this state vector in the standard basis $$\left\{\left\lvert 00 \right\rangle, \left\lvert 01 \right\rangle, \left\lvert 10 \right\rangle, \left\lvert 11 \right\rangle  \right\}.$$
More concretely, the output tells us that after the simulation, we find the system in the state $$\frac{1}{\sqrt{2}} \left( \left\lvert 00 \right\rangle + \left\lvert 11 \right\rangle \right).$$

#### Measurements
Measurement instructions, much like gates, can be part of circuits. The following example prints the output of simulating a measurement performed after passing a single qubit ground state through a Hadamard gate.
The resulting stack of density operators has two entries, corresponding to the two possible outcomes of the measurements. We can read off that the probability of each outcome is one half.

In [12]:
circ_meas = QCirc.FromString("""
H
M
""")

circ_meas.simulate(DOp.GroundState(1))

[(0.5000000000000001,
  array([[1.+0.j, 0.+0.j],
         [0.+0.j, 0.+0.j]])),
 (0.5000000000000001,
  array([[0.+0.j, 0.+0.j],
         [0.+0.j, 1.+0.j]]))]

#### Conditional Layers
Quantum circuits may contain layers that are only executed if a previous measurement yielded a specific result. In the following example, the layer "X?0" will be treated as the layer "X" if the 0th measurement result was 1, and as the identity layer "|" if it was 0. Accordingly, the final state of the system will always be $\left\lvert 0 \right\rangle$.

In [13]:
circ_cond = QCirc.FromString("""
H
M
X?0
""")

result_cond = DOp.MeldStack(circ_cond.simulate(DOp.GroundState(1)))
DOp.MeasureProb(result_cond, 0, 1)

0.0

#### Qubit Teleportation
The Quantum Teleportation algorithm is well-documented. Here, it teleports the state of qubit number 2 into qubit number 0.
The two circuits shown demonstrate this for the initial states $\left\lvert 0 \right\rangle$ and $\left\lvert 1 \right\rangle$ for qubit number 2.

In [14]:
circ_tp_0 = QCirc.FromString("""
| H |
x-o |
| x-o
| | H
| M M
X | |?1
Z | |?0
""")

circ_tp_1 = QCirc.FromString("""
| H X
x-o |
| x-o
| | H
| M M
X | |?1
Z | |?0
""")

result_tp_0 = DOp.MeldStack(circ_tp_0.simulate(DOp.GroundState(3)))
result_tp_1 = DOp.MeldStack(circ_tp_1.simulate(DOp.GroundState(3)))
DOp.MeasureProb(result_tp_0,0,1)

0.0

While the above output demonstrates teleportation of the state $\left\lvert 0 \right\rangle$, the next output shows teleportation of the state $\left\lvert 1 \right\rangle$:

In [15]:
DOp.MeasureProb(result_tp_1,0,1)

1.0000000000000002

#### Grover's algorithm
Grover's algorithm is a quantum unstructured search algorithm. In the example below, an oracle function for the value 110 (binary) is encoded, and then, Grover's Algorithm is applied in the following layers. 
The output shows that indeed, 110 is the most probable measurement result of the 3-qubit system after the simulation.

In [16]:
circ_grover = QCirc.FromString("""
H H H
H | X
L-.-.
H | X
H H H
X X X
H | |
L-.-.
H | |
X X X
H H H
""")

result_grover = DOp.MeldStack(circ_grover.simulate(DOp.GroundState(3)))
[DOp.MeasureProb(result_grover,0,1), DOp.MeasureProb(result_grover,1,1), DOp.MeasureProb(result_grover,2,1)]

[0.8750000000000014, 0.8750000000000014, 0.12500000000000025]

#### Quantum Adder
The quantum adder is a circuit that adds two bits A and B (and possibly, a carried bit from previous addition) together.
To perform the addition 1+0, the first layer has to be changed to "| | X |", and similarly, to "| | | X" for 0+1, and to "| | X X" for 1+1. 

In [27]:
# input: (zero) - (carry) - A - B
circ_adder = QCirc.FromString("""
| | | |
L-+-.-.
| | x-o
L-.-. |
| x-o |
| | x-o
| | | |
""")
# output: carry - sum - A - B

result_adder = DOp.MeldStack(circ_adder.simulate(DOp.GroundState(4)))
print("Sum: " + str(DOp.MeasureProb(result_adder,1,1)) + ", Carrying: " + str(DOp.MeasureProb(result_adder,0,1)))

Sum: 0.0, Carrying: 0.0
